### Stuff Method
In this section we explore retrieval using the `stuff` method.

In [38]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [39]:
response = qa_stuff.run(query)
display(Markdown(response))



> Entering new RetrievalQA chain...



> Finished chain.


Europe sought to align the minds of Africans to the universal and divorce them from their societies

## Other Retrieval Methods

### Map Reduce
Gets a response for each chunk and question pair then uses another llm call to summarize the responses.
Good for recursive document summarization.

- Requires more calls
- Assumes documents (chunks) are independent

In [40]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="map_reduce", 
    retriever=retriever, 
    verbose=True
)

In [42]:
response = qa_stuff.run(query)
display(Markdown(response))



> Entering new RetrievalQA chain...


IndexError: list index out of range

### Refine
Iteratively builds answer from past calls (like a cascade).

- Longer answer
- Takes longer (slower)
- At least as many calls as map reduce.

In [43]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="refine", 
    retriever=retriever, 
    verbose=True
)

In [44]:
response = qa_stuff.run(query)
display(Markdown(response))



> Entering new RetrievalQA chain...


IndexError: list index out of range

### Map Rerank (Experimental)
Single call for each document returning a score from which we specify the highest score to select.
- Faster